In [10]:
import hashlib 
def compute_hash(path, algorithm): 
    h = hashlib.new(algorithm) 
    with open(path, "rb") as f: 
        h.update(f.read()) 
    return h.hexdigest() 
sample = r"C:\Users\jaris\Downloads\ProcessMonitor/Procmon.exe" 
print("MD5:   ", compute_hash(sample, "md5")) 
print("SHA1:  ", compute_hash(sample, "sha1")) 
print("SHA256:", compute_hash(sample, "sha256"))

MD5:    c3e77b6959cc68baee9825c84dc41d9c
SHA1:   bc18a67ad4057dd36f896a4d411b8fc5b06e5b2f
SHA256: 3b7ea4318c3c1508701102cf966f650e04f28d29938f85d74ec0ec2528657b6e


In [3]:
import re 
def extract_strings(path): 
    with open(path, "rb") as f: 
        data = f.read() 
    pattern = rb"[ -~]{4,}" 
    return re.findall(pattern, data) 
strings = extract_strings(sample) 
for s in strings[:20]: 
    print(s.decode(errors="ignore")) 

!This program cannot be run in DOS mode.
V*0T
0RichU
.text
`.rdata
@.data
.rsrc
@.reloc
hpqQ
h`EN
h|nN
h\nN
hlnN
=UUU
h_rM
hDLN
h`GO
hDLN
h|GO
hDLN


In [4]:
import pefile 
pe = pefile.PE(sample) 
print("Entry Point:", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint)) 
print("Image Base:", hex(pe.OPTIONAL_HEADER.ImageBase)) 
print("\nImported DLLs and functions:") 
for entry in pe.DIRECTORY_ENTRY_IMPORT: 
    print(" ", entry.dll.decode()) 
    for imp in entry.imports[:5]: 
        print("     -", imp.name.decode() if imp.name else "None") 

Entry Point: 0xa7f70
Image Base: 0x400000

Imported DLLs and functions:
  WS2_32.dll
     - getsockname
     - listen
     - recv
     - closesocket
     - socket
  VERSION.dll
     - GetFileVersionInfoW
     - VerQueryValueW
     - GetFileVersionInfoSizeW
  COMCTL32.dll
     - ImageList_ReplaceIcon
     - ImageList_SetBkColor
     - ImageList_AddMasked
     - ImageList_BeginDrag
     - ImageList_EndDrag
  FLTLIB.DLL
     - FilterSendMessage
     - FilterGetMessage
     - FilterReplyMessage
     - FilterConnectCommunicationPort
  KERNEL32.dll
     - AcquireSRWLockExclusive
     - AcquireSRWLockShared
     - InitializeSRWLock
     - GetSystemInfo
     - VerSetConditionMask
  USER32.dll
     - LoadStringA
     - DrawEdge
     - GetMessageW
     - TranslateMessage
     - DispatchMessageW
  GDI32.dll
     - SaveDC
     - RestoreDC
     - SetBrushOrgEx
     - SetPixel
     - PatBlt
  COMDLG32.dll
     - ChooseColorW
     - GetOpenFileNameW
     - PrintDlgW
     - ChooseFontW
     - FindText

In [5]:
import yara 
rule_source = """ 
rule ContainsHTTP { 
    strings: 
        $s = "http" 
    condition: 
        $s 
} 
""" 
rules = yara.compile(source=rule_source) 
matches = rules.match(sample) 
print(matches) 


[ContainsHTTP]


In [ ]:
import hashlib, pefile, re, yara 
# sample = "samples/procmon.exe" 
def compute_hashes(path): 
    algos = ["md5", "sha1", "sha256"] 
    output = {} 
    for a in algos: 
        h = hashlib.new(a) 
        with open(path, "rb") as f: 
            h.update(f.read()) 
        output[a] = h.hexdigest() 
    return output 
def extract_strings(path): 
    with open(path, "rb") as f: 
        data = f.read() 
    return re.findall(rb"[ -~]{4,}", data) 
print("Hashes:", compute_hashes(sample)) 
print("\nStrings:") 
print(extract_strings(sample)[:10]) 
print("\nImports:") 
pe = pefile.PE(sample) 
for entry in pe.DIRECTORY_ENTRY_IMPORT: 
    print(entry.dll.decode()) 
print("\nIOCs:") 
decoded = open(sample, "rb").read().decode(errors="ignore") 
print("URLs:", re.findall(r"https?://[^\s\"']+", decoded)) 
print("IPs:", re.findall(r"\b\d{1,3}(?:\.\d{1,3}){3}\b", decoded)) 
print("\nYARA:") 
rule = yara.compile(source=""" 
rule Simple { 
    strings: $s = "http" 
condition: $s 
} 
""") 
print(rule.match(sample))

Hashes: {'md5': 'c3e77b6959cc68baee9825c84dc41d9c', 'sha1': 'bc18a67ad4057dd36f896a4d411b8fc5b06e5b2f', 'sha256': '3b7ea4318c3c1508701102cf966f650e04f28d29938f85d74ec0ec2528657b6e'}

Strings:
[b'!This program cannot be run in DOS mode.', b'V*0T', b'0RichU', b'.text', b'`.rdata', b'@.data', b'.rsrc', b'@.reloc', b'hpqQ', b'h`EN']

Imports:
WS2_32.dll
VERSION.dll
COMCTL32.dll
FLTLIB.DLL
KERNEL32.dll
USER32.dll
GDI32.dll
COMDLG32.dll
ADVAPI32.dll
SHELL32.dll
ole32.dll
OLEAUT32.dll
SHLWAPI.dll
UxTheme.dll
dwmapi.dll
ntdll.dll

IOCs:
URLs: ['https://go.microsoft.com/fwlink/?LinkId=521839', 'https://go.microsoft.com/fwlink/?LinkId=521839', 'https://go.microsoft.com/fwlink/?LinkId=521839\\ul0\\cf0}}}}\\f0\\fs20', 'https://microsoft.com/exporting', 'https://microsoft.com/exporting}}}}\\f0\\fs19', 'http://www.microsoft.com/pkiops/crl/Microsoft%20Windows%20Third%20Party%20Component%20CA%202012.crl0\x06\x08+\x06\x01\x05\x05\x07\x01\x01\x04u0s0q\x06\x08+\x06\x01\x05\x05\x070\x02ehttp://www.microso

In [ ]:
#REFLECTION
# Through this seminar I was able to look at different techniques that are utilized in static malware analysis and how the use of different libraries can be used to uncover information of and executable.
# I understood the importance of simple pattern matching through extracting readable strings from binary. It also helped me understand why one of the first steps taking when examining files is string extraction.
# I also worked with PE headers which gave me an demonstration of how the operating system  workings with a program. I was also shown through imported DLLs how malware relays on specific system libraries.
# I was shown how signatures are with for detection through the creation and application of YARA rules. I was able to see how a simple rule can demonstrate the structure of string matches and conditions which made it clear how real world analysts are able to automate the work of flagging suspicious files.
# Through the collection of simple indicators of compromise I was able to se how regular expressions can expose a users network activity.
# Overall through this seminar, i was able to fortify my understanding on how static analysis takes place and works coupled with how the utilization of different techniques can result in deeper insight of what an executable may contain or attempt to accomplish.
